In [1219]:
import pandas as pd 
import numpy as np 

import warnings
warnings.filterwarnings("ignore")

In [1220]:
df= pd.read_csv("match_data.csv")
df.head()

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,cricsheet_id
0,1370353,2023,2023-05-29,"Narendra Modi Stadium, Ahmedabad",1,0.1,Gujarat Titans,Chennai Super Kings,WP Saha,Shubman Gill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1370353
1,1370353,2023,2023-05-29,"Narendra Modi Stadium, Ahmedabad",1,0.2,Gujarat Titans,Chennai Super Kings,WP Saha,Shubman Gill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1370353
2,1370353,2023,2023-05-29,"Narendra Modi Stadium, Ahmedabad",1,0.3,Gujarat Titans,Chennai Super Kings,WP Saha,Shubman Gill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1370353
3,1370353,2023,2023-05-29,"Narendra Modi Stadium, Ahmedabad",1,0.4,Gujarat Titans,Chennai Super Kings,Shubman Gill,WP Saha,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1370353
4,1370353,2023,2023-05-29,"Narendra Modi Stadium, Ahmedabad",1,0.5,Gujarat Titans,Chennai Super Kings,WP Saha,Shubman Gill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1370353


In [1221]:
df.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'cricsheet_id'],
      dtype='object')

In [1222]:
df= df.drop([ 'ball','non_striker', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type','other_wicket_type','other_player_dismissed', 'cricsheet_id'],axis=1)

In [1223]:
df.sort_values(by='start_date',inplace=True)

In [1224]:
df.rename(columns = {'bowling_team':'opponent_team','striker':'batsman','runs_off_bat':'runs'}, inplace = True) 

In [1225]:
df['season'] = df['season'].replace({'2020/21': '2021', '2009/10': '2009','2007/08': '2008'}).astype(int)

In [1226]:
df = df[df['season']> 2015]

In [1227]:
df.head()

,match_id,season,start_date,venue,innings,batting_team,opponent_team,batsman,bowler,runs,extras,player_dismissed
121166,980901,2016,2016-04-09,Wankhede Stadium,1,Mumbai Indians,Rising Pune Supergiants,S Gopal,M Ashwin,0,0,NaN
121174,980901,2016,2016-04-09,Wankhede Stadium,1,Mumbai Indians,Rising Pune Supergiants,AT Rayudu,M Ashwin,1,0,NaN
121173,980901,2016,2016-04-09,Wankhede Stadium,1,Mumbai Indians,Rising Pune Supergiants,Harbhajan Singh,R Bhatia,0,0,NaN
121172,980901,2016,2016-04-09,Wankhede Stadium,1,Mumbai Indians,Rising Pune Supergiants,AT Rayudu,R Bhatia,1,0,NaN
121176,980901,2016,2016-04-09,Wankhede Stadium,1,Mumbai Indians,Rising Pune Supergiants,Harbhajan Singh,M Ashwin,0,0,NaN


In [1228]:
bowler_stats = df.groupby(['match_id', 'bowler', 'innings'])['runs'].sum().reset_index(name='runs_given')
bowler_stats.head()

,match_id,bowler,innings,runs_given
0,980901,HH Pandya,2,12
1,980901,Harbhajan Singh,2,24
2,980901,I Sharma,1,26
3,980901,JJ Bumrah,2,29
4,980901,M Ashwin,1,16


In [1229]:
wickets_taken_in_inning = df.groupby(['match_id', 'innings', 'bowler'])['player_dismissed'].count().reset_index(name='wickets_taken')
wickets_taken_in_inning.head()

,match_id,innings,bowler,wickets_taken
0,980901,1,I Sharma,2
1,980901,1,M Ashwin,1
2,980901,1,MR Marsh,2
3,980901,1,R Ashwin,1
4,980901,1,R Bhatia,1


In [1230]:
bowler_stats = pd.merge(bowler_stats,wickets_taken_in_inning,on=['match_id', 'innings', 'bowler'])
bowler_stats.head()

,match_id,bowler,innings,runs_given,wickets_taken
0,980901,HH Pandya,2,12,0
1,980901,Harbhajan Singh,2,24,1
2,980901,I Sharma,1,26,2
3,980901,JJ Bumrah,2,29,0
4,980901,M Ashwin,1,16,1


In [1231]:
# # Bowling Average Against Opponent
# bowling_avg = df.groupby(['bowler', 'opponent_team']).apply(lambda x: x['runs'].sum() / x['player_dismissed'].count()).reset_index(name='bowling_avg')

# # Economy Rate Against Opponent
# economy_rate = df.groupby(['bowler', 'opponent_team']).apply(lambda x: x['runs'].sum() / (x['extras'].sum() + x['runs'].sum())).reset_index(name='economy_rate')

# # Bowling Strike Rate Against Opponent
# bowling_strike_rate = df.groupby(['bowler', 'opponent_team']).apply(lambda x: (x['balls'].sum() / x['player_dismissed'].count()) * 100).reset_index(name='bowling_strike_rate')

# # Percentage of Matches with 3+ Wickets Against Opponent
# matches_with_3plus_wickets = df[df['player_dismissed'].notna()].groupby(['bowler', 'opponent_team']).apply(lambda x: (x['player_dismissed'].count() >= 3).sum() / x['match_id'].nunique()).reset_index(name='matches_with_3plus_wickets')

# # Average Wickets Taken per Match Against Opponent
# avg_wickets_per_match = df[df['player_dismissed'].notna()].groupby(['bowler', 'opponent_team']).apply(lambda x: x['player_dismissed'].count() / x['match_id'].nunique()).reset_index(name='avg_wickets_per_match')


In [1232]:
import pandas as pd

# Calculate batsman-level statistics
batsman_stats = df.groupby(['match_id', 'batsman', 'innings','batting_team']).agg({
    'opponent_team':'first',
    'venue':'first',
    'runs': 'sum',
    'player_dismissed': lambda x: max(x.count() - x.isnull().sum(), 0)
}).reset_index()

# Calculate balls faced
balls_faced = df.groupby(['match_id', 'batsman', 'innings', 'batting_team'])['runs'].count().reset_index()
balls_faced.columns = ['match_id', 'batsman', 'innings', 'batting_team', 'balls_faced']


# Calculate dot balls
dot_balls = df.groupby(['match_id', 'batsman', 'innings', 'batting_team']).apply(lambda x: (x['runs'] == 0).sum()).reset_index()
dot_balls.columns = ['match_id', 'batsman', 'innings', 'batting_team', 'dot_balls']

# Combine balls_faced and dot_balls DataFrames
combined_stats = pd.merge(balls_faced, dot_balls, on=['match_id', 'batsman', 'innings', 'batting_team'])

# Calculate total runs scored by the team in each inning of each match
team_total_runs = df.groupby(['match_id', 'innings', 'batting_team'])['runs'].sum().reset_index()
team_total_runs.rename(columns={'runs': 'total_runs_team'}, inplace=True)


# Merge team total runs with batsman statistics
batsman_stats = pd.merge(batsman_stats, team_total_runs, how='left', on=['match_id', 'innings', 'batting_team'])

batsman_stats = pd.merge(batsman_stats, combined_stats, how='left', on=['match_id','batsman', 'innings', 'batting_team'])

# batsman_stats['Batting_Average'] = batsman_stats['runs'] / batsman_stats['player_dismissed']
batsman_stats['50+ runs'] = (batsman_stats['runs'] >= 50).astype(int)
batsman_stats['100+ runs'] = (batsman_stats['runs'] >= 100).astype(int)
batsman_stats['strike_rate'] = (batsman_stats['runs'] / batsman_stats['balls_faced']) * 100
batsman_stats['batting_contribution'] = (batsman_stats['runs'] / batsman_stats['total_runs_team'])

# Select relevant columns for the final DataFrame
# final_df = batsman_stats[['match_id', 'batsman', 'innings', 'runs', 'total_runs_team', '50+ runs', '100+ runs','dot_balls','player_dismissed', 'strike_rate', 'batting_Contribution']]
final_df = batsman_stats

# Display the final DataFrame
final_df.head()


,match_id,batsman,innings,batting_team,opponent_team,venue,runs,player_dismissed,total_runs_team,balls_faced,dot_balls,50+ runs,100+ runs,strike_rate,batting_contribution
0,980901,AM Rahane,2,Rising Pune Supergiants,Mumbai Indians,Wankhede Stadium,66,0,121,42,16,1,0,157.142857,0.545455
1,980901,AT Rayudu,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,22,0,108,28,12,0,0,78.571429,0.203704
2,980901,F du Plessis,2,Rising Pune Supergiants,Mumbai Indians,Wankhede Stadium,34,0,121,36,21,0,0,94.444444,0.280992
3,980901,HH Pandya,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,9,0,108,13,10,0,0,69.230769,0.083333
4,980901,Harbhajan Singh,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,45,0,108,31,14,0,0,145.161290,0.416667


In [1233]:
len(final_df)

7686

In [1234]:
match_result = final_df.groupby(['match_id','batting_team'])['total_runs_team'].first().reset_index()

# Group the DataFrame by match_id
grouped = match_result.groupby('match_id')

max_runs_per_match = grouped['total_runs_team'].transform('max')

# Determine win or loss for each team in each match
match_result['result'] = match_result['total_runs_team'] == max_runs_per_match

# Map boolean results to 'win' or 'loss'
match_result['result'] = match_result['result'].map({True: 1, False: 0})

# Print the updated DataFrame
match_result.head()


,match_id,batting_team,total_runs_team,result
0,980901,Mumbai Indians,108,0
1,980901,Rising Pune Supergiants,121,1
2,980903,Delhi Daredevils,86,0
3,980903,Kolkata Knight Riders,88,1
4,980905,Gujarat Lions,156,1


In [1235]:
final_df = pd.merge(final_df, match_result, how='left', on=['match_id', 'batting_team','total_runs_team'])
final_df.head()


,match_id,batsman,innings,batting_team,opponent_team,venue,runs,player_dismissed,total_runs_team,balls_faced,dot_balls,50+ runs,100+ runs,strike_rate,batting_contribution,result
0,980901,AM Rahane,2,Rising Pune Supergiants,Mumbai Indians,Wankhede Stadium,66,0,121,42,16,1,0,157.142857,0.545455,1.0
1,980901,AT Rayudu,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,22,0,108,28,12,0,0,78.571429,0.203704,0.0
2,980901,F du Plessis,2,Rising Pune Supergiants,Mumbai Indians,Wankhede Stadium,34,0,121,36,21,0,0,94.444444,0.280992,1.0
3,980901,HH Pandya,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,9,0,108,13,10,0,0,69.230769,0.083333,0.0
4,980901,Harbhajan Singh,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,45,0,108,31,14,0,0,145.161290,0.416667,0.0


In [1236]:
weights = {
    'runs': 0.5,
    '50+ runs': 0.1,
    '100+ runs': 0.1,
    'dot_balls': 0.1,
    'strike_rate': 0.1,
    'batting_Contribution':0.1
}

In [1237]:
weighted_score = final_df[['runs', '50+ runs', '100+ runs', 'dot_balls', 'strike_rate', 'batting_contribution']].multiply(weights, axis=1).sum(axis=1)

# Add the weighted score as a new column
final_df['player_strength'] = weighted_score
final_df.head()

,match_id,batsman,innings,batting_team,opponent_team,venue,runs,player_dismissed,total_runs_team,balls_faced,dot_balls,50+ runs,100+ runs,strike_rate,batting_contribution,result,player_strength
0,980901,AM Rahane,2,Rising Pune Supergiants,Mumbai Indians,Wankhede Stadium,66,0,121,42,16,1,0,157.142857,0.545455,1.0,50.414286
1,980901,AT Rayudu,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,22,0,108,28,12,0,0,78.571429,0.203704,0.0,20.057143
2,980901,F du Plessis,2,Rising Pune Supergiants,Mumbai Indians,Wankhede Stadium,34,0,121,36,21,0,0,94.444444,0.280992,1.0,28.544444
3,980901,HH Pandya,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,9,0,108,13,10,0,0,69.230769,0.083333,0.0,12.423077
4,980901,Harbhajan Singh,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,45,0,108,31,14,0,0,145.161290,0.416667,0.0,38.416129


In [1238]:
final_df.columns

Index(['match_id', 'batsman', 'innings', 'batting_team', 'opponent_team',
       'venue', 'runs', 'player_dismissed', 'total_runs_team', 'balls_faced',
       'dot_balls', '50+ runs', '100+ runs', 'strike_rate',
       'batting_contribution', 'result', 'player_strength'],
      dtype='object')

In [1239]:
df2 = final_df[['match_id', 'batsman', 'innings', 'batting_team', 'opponent_team',
       'venue','runs','50+ runs', '100+ runs','player_strength','result']]
df2.head()

,match_id,batsman,innings,batting_team,opponent_team,venue,runs,50+ runs,100+ runs,player_strength,result
0,980901,AM Rahane,2,Rising Pune Supergiants,Mumbai Indians,Wankhede Stadium,66,1,0,50.414286,1.0
1,980901,AT Rayudu,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,22,0,0,20.057143,0.0
2,980901,F du Plessis,2,Rising Pune Supergiants,Mumbai Indians,Wankhede Stadium,34,0,0,28.544444,1.0
3,980901,HH Pandya,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,9,0,0,12.423077,0.0
4,980901,Harbhajan Singh,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,45,0,0,38.416129,0.0


In [1240]:
# venue related features
df2['50+ on ground'] = df2.groupby(['batsman','venue'])['50+ runs'].transform('sum')
df2['100+ on ground'] = df2.groupby(['batsman','venue'])['100+ runs'].transform('sum')
df2['avg_runs_ground'] = df2.groupby(['batsman','venue'])['runs'].transform('mean')
df2.head()

,match_id,batsman,innings,batting_team,opponent_team,venue,runs,50+ runs,100+ runs,player_strength,result,50+ on ground,100+ on ground,avg_runs_ground
0,980901,AM Rahane,2,Rising Pune Supergiants,Mumbai Indians,Wankhede Stadium,66,1,0,50.414286,1.0,2,0,46.800000
1,980901,AT Rayudu,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,22,0,0,20.057143,0.0,0,0,15.888889
2,980901,F du Plessis,2,Rising Pune Supergiants,Mumbai Indians,Wankhede Stadium,34,0,0,28.544444,1.0,1,0,37.000000
3,980901,HH Pandya,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,9,0,0,12.423077,0.0,0,0,16.120000
4,980901,Harbhajan Singh,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,45,0,0,38.416129,0.0,0,0,8.666667


In [1241]:
# opponent related features
df2['avg_runs_to_opponent'] = df2.groupby(['batsman','opponent_team'])['runs'].transform('mean')
df2['50+_runs_against_opponent'] = df2.groupby(['batsman','opponent_team'])['50+ runs'].transform('sum')
df2['100+_runs_against_opponent'] = df2.groupby(['batsman','opponent_team'])['100+ runs'].transform('sum')
# df2['recent_avg_runs_opponent'] = df2.groupby(['batsman', 'opponent_team'])['runs'].tail(5).groupby([df2['batsman'], df2['opponent_team']]).mean().reset_index(drop=True)

df2.head()

,match_id,batsman,innings,batting_team,opponent_team,venue,runs,50+ runs,100+ runs,player_strength,result,50+ on ground,100+ on ground,avg_runs_ground,avg_runs_to_opponent,50+_runs_against_opponent,100+_runs_against_opponent
0,980901,AM Rahane,2,Rising Pune Supergiants,Mumbai Indians,Wankhede Stadium,66,1,0,50.414286,1.0,2,0,46.800000,29.352941,4,0
1,980901,AT Rayudu,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,22,0,0,20.057143,0.0,0,0,15.888889,22.000000,0,0
2,980901,F du Plessis,2,Rising Pune Supergiants,Mumbai Indians,Wankhede Stadium,34,0,0,28.544444,1.0,1,0,37.000000,32.900000,4,0
3,980901,HH Pandya,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,9,0,0,12.423077,0.0,0,0,16.120000,9.000000,0,0
4,980901,Harbhajan Singh,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,45,0,0,38.416129,0.0,0,0,8.666667,45.000000,0,0


In [1242]:
vk_df =df2[(df2['batsman']=='V Kohli') & (df2['opponent_team']=='Kolkata Knight Riders')].sort_values(by='player_strength',ascending=False)
vk_df.head()

,match_id,batsman,innings,batting_team,opponent_team,venue,runs,50+ runs,100+ runs,player_strength,result,50+ on ground,100+ on ground,avg_runs_ground,avg_runs_to_opponent,50+_runs_against_opponent,100+_runs_against_opponent
3186,1178410,V Kohli,1,Royal Challengers Bangalore,Kolkata Knight Riders,Eden Gardens,100,1,1,68.641379,1.0,2,1,51.500000,37.625,6,1
2927,1175372,V Kohli,1,Royal Challengers Bangalore,Kolkata Knight Riders,M.Chinnaswamy Stadium,84,1,0,60.042857,1.0,3,0,33.714286,37.625,6,1
669,980995,V Kohli,2,Royal Challengers Bangalore,Kolkata Knight Riders,Eden Gardens,75,1,0,53.605882,1.0,2,1,51.500000,37.625,6,1
2216,1136589,V Kohli,1,Royal Challengers Bangalore,Kolkata Knight Riders,M.Chinnaswamy Stadium,68,1,0,50.754545,0.0,3,0,33.714286,37.625,6,1
7109,1359510,V Kohli,2,Royal Challengers Bangalore,Kolkata Knight Riders,"M Chinnaswamy Stadium, Bengaluru",54,1,0,41.910526,0.0,5,1,50.571429,37.625,6,1


In [1243]:
df2.head()#sort_values(by='player_strength',ascending=False).head()

,match_id,batsman,innings,batting_team,opponent_team,venue,runs,50+ runs,100+ runs,player_strength,result,50+ on ground,100+ on ground,avg_runs_ground,avg_runs_to_opponent,50+_runs_against_opponent,100+_runs_against_opponent
0,980901,AM Rahane,2,Rising Pune Supergiants,Mumbai Indians,Wankhede Stadium,66,1,0,50.414286,1.0,2,0,46.800000,29.352941,4,0
1,980901,AT Rayudu,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,22,0,0,20.057143,0.0,0,0,15.888889,22.000000,0,0
2,980901,F du Plessis,2,Rising Pune Supergiants,Mumbai Indians,Wankhede Stadium,34,0,0,28.544444,1.0,1,0,37.000000,32.900000,4,0
3,980901,HH Pandya,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,9,0,0,12.423077,0.0,0,0,16.120000,9.000000,0,0
4,980901,Harbhajan Singh,1,Mumbai Indians,Rising Pune Supergiants,Wankhede Stadium,45,0,0,38.416129,0.0,0,0,8.666667,45.000000,0,0


In [1244]:
df2.columns

Index(['match_id', 'batsman', 'innings', 'batting_team', 'opponent_team',
       'venue', 'runs', '50+ runs', '100+ runs', 'player_strength', 'result',
       '50+ on ground', '100+ on ground', 'avg_runs_ground',
       'avg_runs_to_opponent', '50+_runs_against_opponent',
       '100+_runs_against_opponent'],
      dtype='object')

In [1245]:
player_df = df2.groupby(['batsman','opponent_team','venue'])['player_strength'].mean().reset_index()
player_df.head()

,batsman,opponent_team,venue,player_strength
0,A Ashish Reddy,Kolkata Knight Riders,"Rajiv Gandhi International Stadium, Uppal",21.144444
1,A Ashish Reddy,Royal Challengers Bangalore,M Chinnaswamy Stadium,34.477778
2,A Ashish Reddy,Royal Challengers Bangalore,"Rajiv Gandhi International Stadium, Uppal",7.766667
3,A Badoni,Chennai Super Kings,Bharat Ratna Shri Atal Bihari Vajpayee Ekana C...,47.457143
4,A Badoni,Chennai Super Kings,"Brabourne Stadium, Mumbai",23.771429


In [1246]:
df3 = df2[[ 'batsman', 'opponent_team','venue','50+ on ground', '100+ on ground', 'avg_runs_ground',
       'avg_runs_to_opponent', '50+_runs_against_opponent',
       '100+_runs_against_opponent']]
df3.head()

,batsman,opponent_team,venue,50+ on ground,100+ on ground,avg_runs_ground,avg_runs_to_opponent,50+_runs_against_opponent,100+_runs_against_opponent
0,AM Rahane,Mumbai Indians,Wankhede Stadium,2,0,46.800000,29.352941,4,0
1,AT Rayudu,Rising Pune Supergiants,Wankhede Stadium,0,0,15.888889,22.000000,0,0
2,F du Plessis,Mumbai Indians,Wankhede Stadium,1,0,37.000000,32.900000,4,0
3,HH Pandya,Rising Pune Supergiants,Wankhede Stadium,0,0,16.120000,9.000000,0,0
4,Harbhajan Singh,Rising Pune Supergiants,Wankhede Stadium,0,0,8.666667,45.000000,0,0


In [1247]:
df3.drop_duplicates(inplace=True)

In [1248]:
df3=pd.merge(df3,player_df,on=['batsman', 'opponent_team','venue'])
df3.head()

,batsman,opponent_team,venue,50+ on ground,100+ on ground,avg_runs_ground,avg_runs_to_opponent,50+_runs_against_opponent,100+_runs_against_opponent,player_strength
0,AM Rahane,Mumbai Indians,Wankhede Stadium,2,0,46.800000,29.352941,4,0,38.275152
1,AT Rayudu,Rising Pune Supergiants,Wankhede Stadium,0,0,15.888889,22.000000,0,0,20.057143
2,F du Plessis,Mumbai Indians,Wankhede Stadium,1,0,37.000000,32.900000,4,0,28.544444
3,HH Pandya,Rising Pune Supergiants,Wankhede Stadium,0,0,16.120000,9.000000,0,0,12.423077
4,Harbhajan Singh,Rising Pune Supergiants,Wankhede Stadium,0,0,8.666667,45.000000,0,0,38.416129


In [1249]:
df3.columns

Index(['batsman', 'opponent_team', 'venue', '50+ on ground', '100+ on ground',
       'avg_runs_ground', 'avg_runs_to_opponent', '50+_runs_against_opponent',
       '100+_runs_against_opponent', 'player_strength'],
      dtype='object')

In [1250]:
df[df['season']==2023].batting_team.value_counts().reset_index()

,batting_team,count
0,Gujarat Titans,2050
1,Mumbai Indians,1925
2,Lucknow Super Giants,1832
3,Chennai Super Kings,1804
4,Punjab Kings,1752
5,Kolkata Knight Riders,1728
6,Sunrisers Hyderabad,1728
7,Delhi Capitals,1718
8,Royal Challengers Bangalore,1693
9,Rajasthan Royals,1633


In [1251]:
# no_of_dismiss = batsman_stats.groupby('batsman')['player_dismissed'].sum().reset_index()
# overall_runs = batsman_stats.groupby('batsman')['runs'].sum().reset_index()

# batting_average = pd.merge(overall_runs, no_of_dismiss, on='batsman')
# batting_average['batting_average'] = batting_average['runs'] / batting_average['player_dismissed']\

# batting_average.head()


In [1252]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler

# Split the data into features (X) and target variable (y)
X = df3[['batsman', 'opponent_team', 'venue', '50+ on ground', '100+ on ground',
       'avg_runs_ground', 'avg_runs_to_opponent', '50+_runs_against_opponent',
       '100+_runs_against_opponent']]
y = df3['player_strength']

label_encoder = LabelEncoder()
X['batsman'] = label_encoder.fit_transform(X['batsman'])
X['opponent_team'] = label_encoder.fit_transform(X['opponent_team'])
X['venue'] = label_encoder.fit_transform(X['venue'])


print(X.head())
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
# Fit the scaler on the training data and transform both training and testing data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and train the regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)



   batsman  opponent_team  venue  50+ on ground  100+ on ground  \
0       22              8     34              2               0   
1       30             12     34              0               0   
2       99              8     34              1               0   
3      114             12     34              0               0   
4      118             12     34              0               0   

   avg_runs_ground  avg_runs_to_opponent  50+_runs_against_opponent  \
0        46.800000             29.352941                          4   
1        15.888889             22.000000                          0   
2        37.000000             32.900000                          4   
3        16.120000              9.000000                          0   
4         8.666667             45.000000                          0   

   100+_runs_against_opponent  
0                           0  
1                           0  
2                           0  
3                           0  
4         

In [1253]:
# import joblib

# # Assuming 'model' is your trained model
# joblib.dump(class_model, 'player_win_model.pkl')

In [1254]:
# df4.to_csv('player_strength_data.csv')

In [1255]:
len(df4)

7648

In [1256]:
df4 =df2[['match_id', 'batsman', 'innings', 'opponent_team','venue', 'result']]
df4.head()

,match_id,batsman,innings,opponent_team,venue,result
0,980901,AM Rahane,2,Mumbai Indians,Wankhede Stadium,1.0
1,980901,AT Rayudu,1,Rising Pune Supergiants,Wankhede Stadium,0.0
2,980901,F du Plessis,2,Mumbai Indians,Wankhede Stadium,1.0
3,980901,HH Pandya,1,Rising Pune Supergiants,Wankhede Stadium,0.0
4,980901,Harbhajan Singh,1,Rising Pune Supergiants,Wankhede Stadium,0.0


In [1257]:
df4= pd.merge(df4,df3,how='left',on=['batsman','opponent_team','venue'])
df4.head()

,match_id,batsman,innings,opponent_team,venue,result,50+ on ground,100+ on ground,avg_runs_ground,avg_runs_to_opponent,50+_runs_against_opponent,100+_runs_against_opponent,player_strength
0,980901,AM Rahane,2,Mumbai Indians,Wankhede Stadium,1.0,2,0,46.800000,29.352941,4,0,38.275152
1,980901,AT Rayudu,1,Rising Pune Supergiants,Wankhede Stadium,0.0,0,0,15.888889,22.000000,0,0,20.057143
2,980901,F du Plessis,2,Mumbai Indians,Wankhede Stadium,1.0,1,0,37.000000,32.900000,4,0,28.544444
3,980901,HH Pandya,1,Rising Pune Supergiants,Wankhede Stadium,0.0,0,0,16.120000,9.000000,0,0,12.423077
4,980901,Harbhajan Singh,1,Rising Pune Supergiants,Wankhede Stadium,0.0,0,0,8.666667,45.000000,0,0,38.416129


In [1258]:
df4= df4.dropna()

In [1259]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

# Split the data into features (X) and target variable (y)
X = df4[['batsman','innings', 'opponent_team', 'venue', '50+ on ground', '100+ on ground',
       'avg_runs_ground', 'avg_runs_to_opponent', '50+_runs_against_opponent',
       '100+_runs_against_opponent','player_strength']]

y = df4['result']

label_encoder = LabelEncoder()
X['batsman'] = label_encoder.fit_transform(X['batsman'])
X['opponent_team'] = label_encoder.fit_transform(X['opponent_team'])
X['venue'] = label_encoder.fit_transform(X['venue'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
# Fit the scaler on the training data and transform both training and testing data
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# Initialize the logistic regression model
class_model = RandomForestClassifier()

# Train the logistic regression model on the training data
class_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = class_model.predict(X_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.6405228758169934


Prediction

In [1260]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
import joblib

input_data = {
    'batsman': 'V Kohli',
    'opponent_team': 'Mumbai Indians',
    'venue': 'Wankhede Stadium',
    'innings':1
}

input_df = pd.DataFrame([input_data])

data = pd.read_csv('player_strength_data.csv')
input_df = pd.merge(input_df,data,on=['batsman','opponent_team','venue'])

inputs = input_df[['batsman','innings', 'opponent_team', 'venue', '50+ on ground', '100+ on ground',
       'avg_runs_ground', 'avg_runs_to_opponent', '50+_runs_against_opponent',
       '100+_runs_against_opponent','player_strength']]

label_encoder = LabelEncoder()
inputs['batsman'] = label_encoder.fit_transform(inputs['batsman'])
inputs['opponent_team'] = label_encoder.fit_transform(inputs['opponent_team'])
inputs['venue'] = label_encoder.fit_transform(inputs['venue'])

print(inputs)

model = joblib.load('player_win_model.pkl')

predict = model.predict_proba(inputs)

print("prediction:", predict)



   batsman  innings  opponent_team  venue  50+ on ground  100+ on ground  \
0        0        1              0      0              1               0   

   avg_runs_ground  avg_runs_to_opponent  50+_runs_against_opponent  \
0            38.25                 33.25                          4   

   100+_runs_against_opponent  player_strength  
0                           0        31.594481  
prediction: [[0.599 0.401]]
